In [1]:
import numpy as np
import math

Entropy

In [2]:

sm = (((0.6)*math.log(0.6,2)) + ((0.4)*math.log(0.4,2)))*(-1)
print(sm)

0.9709505944546686


Information gain

In [ ]:
x10e = ((((0.75)*math.log(0.75,2)) + ((0.25)*math.log(0.25,2)))*-1)
x11e = ((((0.5)*math.log(0.5,2)) + ((0.5)*math.log(0.5,2)))*-1)
print(f"X1 information gain = {(sm-x10e)*0.4 + (sm-x11e)*0.6}")

Neural Network

In [3]:
def predict(w,b,x):
    hret = []
    for i in w:
        h = i[0]
        for j in i[1:]:
            h+= x*j
        h = h if h >0 else 0
        hret.append(h)
    pred = b[0]
    for i,j in zip(hret,b[1:]):
        pred+=i*j
    return 1 if pred >0.5 else -1

def error_rate(w,b,xls, yls):
    c = 0
    for x,y in zip(xls, yls):
        xpred = predict(w,b,x)
        if xpred != y:
            c+= 1
    print(f"{c}/{len(xls)}, or {c/len(xls)}")
    

In [4]:
w =[ [ -1 , -1 ] , [ 0 , 2 ], [ -2 , 1]]
b = [-1,-1,1,-3]
x = [-4,-2,1,2,3,6,7]
y = [1,-1,1,-1,1,-1,-1]
error_rate(w,b,x,y)

2/7, or 0.2857142857142857


Markov Reward Processes

In [28]:
class MRP:
    def __init__(self, states: list, rewards: dict, transitions: dict, gamma: float) -> None:
        self.states = sorted(states)
        self.rewards = rewards
        self.transitions = transitions
        self.gamma = gamma

    def expectedRewards(self, seq_l: int) -> np.ndarray:
        if seq_l < 0:
            raise Exception("l must be >= 0")
        expected_rewards = np.zeros((seq_l+1,len(self.states)))

        for l in range(1, seq_l+1):
            for si, state in enumerate(self.states):
                future_expected_reward = sum([t_prob*expected_rewards[l-1][self.states.index(t_state)] for t_state, t_prob in self.transitions[state]])
                expected_rewards[l][si] = self.rewards[state] + self.gamma * future_expected_reward

        return expected_rewards
    
    def expectedReward(self, seq_l: int, state: str) -> None:
        print("J({},{}) = {}".format(seq_l, state, self.expectedRewards(seq_l)[seq_l][self.states.index(state)]))
    
    def displayRewardTable(self, seq_l: int) -> None:
        expected_rewards = self.expectedRewards(seq_l)
        max_state_len = max([len(s) for s in self.states])
        header = (" " * (max_state_len+6)) + "       ".join([str(l) for l in range(seq_l+1)])
        print(header)
        for state in self.states:
            line = " {} ".format(state.rjust(max_state_len, " ")) + "  ".join(["{:6.3f}".format(expected_rewards[l][self.states.index(state)]) for l in range(seq_l+1)])
            print(line)
        

    
def parseMRPInput(input_lines: list) -> MRP:
    states = []
    rewards = {}
    transitions = {}
    gamma = 0
    for line in input_lines:
        split_line = line.split()
        if len(split_line) == 1:
            gamma = float(split_line[0])
            break
        state = split_line[0]
        reward = float(split_line[1])
        transition = [tuple([split_line[i], float(split_line[i+1])]) for i in range(2,len(split_line),2)]
        states.append(state)
        rewards[state] = reward
        transitions[state] = transition

    return MRP(states, rewards, transitions, gamma)


In [29]:
sample_mrp_input = [
    "A -2 A 0.5 B 0.5",
    "B 0 C 1.0",
    "C 6 A .666 B .333",
    "0.5"
]

sampleMRP = parseMRPInput(sample_mrp_input)
sampleMRP.expectedReward(3, "C")
sampleMRP.displayRewardTable(3)

J(3,C) = 5.667
       0       1       2       3
 A  0.000  -2.000  -2.500  -1.875
 B  0.000   0.000   3.000   2.667
 C  0.000   6.000   5.334   5.667
